# Optional- Common mistakes in python

As you start writing mroe advanced programs, there might be times when Python behaves contrary to expectations and it results in your programs not working. This notebook outlines some common mistakes that people often encounter in their programming careers.

### Mutable default arguments

This was touched upon in lectures, but default arguments with mutable objects are (usually) a bad idea. This is because you can have something like the following example:

In [2]:
def append_3(list_=[]):
    list_.append(3)
    return list_

list1 = append_3()
print("list1 is:", list1)
list2 = append_3()
print("list 2 is:", list2)
print("And list 1 is now:", list1)

list1 is: [3]
list 2 is: [3, 3]
And list 1 is now: [3, 3]


The expected behaviour would probably be the following output:
    
    list1 is: [3]
    list 2 is: [3]
    And list 1 is now: [3]
    
The reason this happens is because the default arguments are evaluated once every time the function is defined, then stored. So if you mutate a default argument, you've mutated it for any future calls to the function. To avoid this, we can do the following:

In [3]:
def append_3(list_=None):
    if list_ is None:
        list_ = []
    list_.append(3)
    return list_

list1 = append_3()
print("list1 is:", list1)
list2 = append_3()
print("list 2 is:", list2)
print("And list 1 is now:", list1)

list1 is: [3]
list 2 is: [3]
And list 1 is now: [3]


Although the code is clunky, it's what we have to do if we want to use Python.

### Late binding closures

This will only really be relevant if you've gone through the Optional notebook on Lambda functions, functions as variables, and decorators. Skip to the next point if you haven't gone through the notebook. Imagine we have the following case:

In [8]:
# Code adapted from http://docs.python-guide.org/en/latest/writing/gotchas/
def create_multipliers(n):
    return [lambda x : i * x for i in range(0,n)]

for multiplier in create_multipliers(5):
    print(multiplier(2))

8
8
8
8
8


That's weird. You porbably expected the following:

This happens because any variable in a Python function is late binding, meaning that the variables are only looked up when the function is actually called. Therefore, i is always 5 from the point of view of the functions in the list. (This happens not only with lambda functions but also with normal functions, if we do a similar thing.)

To avoid this, we can use the fact that default arguments are evaluated at the point of creation:

In [9]:
# Code adapted from http://docs.python-guide.org/en/latest/writing/gotchas/
def create_multipliers(n):
    return [lambda x, i=i : i * x for i in range(0,n)]

for multiplier in create_multipliers(5):
    print(multiplier(2))

0
2
4
6
8


Even if you never mean to use the optional argument, do this so that your code works. Two "wrongs" do make a "right" in this case.

### Writing performance critical code in Python

Don't.

### Truth value of arrays

When using default arguments for list-like objects, if you followed the tips above, you'll probably use None. There's a slight problem with that though if we follow a naïve approach, when dealing with numpy arrays: 

In [4]:
import numpy as np

def append_3(arr_=None):
    if arr_:
        arr_=np.append(arr_,3)
    else:
        arr_ = np.array([])
        arr_ = np.append(arr_, [3,4])
    return arr_

list1 = append_3()
print("list1 is:", list1)
list2 = append_3(list1)
print("list2 is:", list2)
print("And list 1 is now:", list1)

list1 is: [ 3.  4.]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

This is because NumPy isn't sure if you want to check whether the array as a whole is None or not, or whether some elements of the array are None. To get around this, you can do the following:

In [5]:
import numpy as np

def append_3(arr_=None):
    if arr_ is not None:
        arr_=np.append(arr_,3)
    else:
        arr_ = np.array([])
        arr_ = np.append(arr_, [3,4])
    return arr_

list1 = append_3()
print("list1 is:", list1)
list2 = append_3(list1)
print("list2 is:", list2)
print("And list 1 is now:", list1)

list1 is: [ 3.  4.]
list2 is: [ 3.  4.  3.]
And list 1 is now: [ 3.  4.]


### Forgetting your self

It is easy to write code that spits out errors because you forgot to write selfs when you were accessing object properties in methods.

### 0 is False

Consider the following code:

In [7]:
def add_3(to=None):
    if to:
        return to+3.
    else:
        return "What are you doing?"
    
print(add_3(to=0))

What are you doing?


You may have expected the last line to output 3, instead of the string "What are you doing?". This is because although you think you are testing for whether the argument to has been given or not, the if statement is only testing the truth value of the argument, and False, None, and 0 are all false. To avoid this, do the following:

In [8]:
def add_3(to=None):
    if to is not None:
        return to+3.
    else:
        return "What are you doing?"
    
print(add_3(to=0))

3.0
